<a href="https://colab.research.google.com/github/toshNaik/English-German/blob/master/Hallo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Dropout, Flatten, LSTM, Input, Embedding, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import metrics

In [2]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units):
    super(Encoder, self).__init__()
    self.enc_units = enc_units
    self.input_layer = Input(shape=(None, ))
    self.embedding = Embedding(vocab_size, embedding_dim)
    self.lstm = LSTM(self.enc_units, return_state = True)
  
  def call(self, X):
    # X = self.input_layer(X)
    X = self.embedding(X)
    output, state_h, state_c = self.lstm(X)
    states = [state_h, state_c]
    return output, states

In [3]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units):
    super(Decoder, self).__init__()
    self.dec_units = dec_units
    self.input_layer = Input(shape=(None, ))
    self.embedding = Embedding(vocab_size, embedding_dim)
    self.lstm = LSTM(self.dec_units, return_sequences=True, return_state = True)
    self.fc = TimeDistributed(Dense(vocab_size, activation='softmax'))

  def call(self, X, enc_state):
    # X = self.input_layer(X)
    X = self.embedding(X)
    output, state_h, state_c = self.lstm(X, initial_state = enc_state)
    output = self.fc(output)
    states = [state_h, state_c]
    return output, states

In [4]:
class Seq2seq(tf.keras.Model):
  def __init__(self, encoder, decoder):
    super(Seq2seq, self).__init__()
    
    assert encoder.enc_units == decoder.dec_units, "encoder units and decoder units must be same"
    
    self.encoder = encoder
    self.decoder = decoder

  def call(self, X, training=False):
    X_source = X[0]
    X_target = X[1]
    _, encoder_states = self.encoder(X_source)
    decoder_output, _ = self.decoder(X_target, encoder_states)
    return decoder_output
    
  
  def predict(self, X, tokenizer):
    _, init_states = self.encoder(X)
    decoder_init = tokenizer.word_index['bos'] # token of <BOS> will be first input
    decoder_init = np.array([[decoder_init]]) # Input has to be 2d array

    stop = False
    deu_sentence = ''
    
    while not stop:
      output, next_states = self.decoder(decoder_init, init_states)
      new_id = np.argmax(output[0, -1, :])
      new_word = tokenizer.index_word[new_id]
      
      if (new_word == 'eos' or len(deu_sentence.split()) == 77):
        stop = True
      else:
        deu_sentence += new_word + ' '

      dec_word = np.array([[new_id]])
      init_states = next_states
    return deu_sentence

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
!unzip drive/My\ Drive/english-german/deu-eng.zip

Archive:  drive/My Drive/english-german/deu-eng.zip
  inflating: deu.txt                 
  inflating: _about.txt              


In [7]:
file1 = open('deu.txt', 'r')
dataset = []
for line in file1.readlines():
  dataset.append(line)
file1.close()

eng = []
deu = []
for line in dataset:
  english, deutsche, _ = re.split('\t', line)
  english = re.sub(r'[^\w\s]', '', english)   #remove punctuation
  deutsche = re.sub(r'[^\w\s]', '', deutsche)
  english = re.sub(r'\d', '', english)        #remove digits
  deutsche = re.sub(r'\d', '', deutsche)
  eng.append(english.lower())
  deu.append(deutsche.lower()+' <EOS>')

eng_data = pd.Series(eng)
deu_data = pd.Series(deu)
deu_input_data = deu_data.copy().apply(lambda x: '<BOS>' + x)

def tokenize(data):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(data)
  return tokenizer

english_tokenizer = tokenize(eng_data.values)
english_vocab_size = len(english_tokenizer.word_index) + 1
print(f'English vocab size {english_vocab_size}')

german_tokenizer = tokenize(deu_input_data.values)
german_vocab_size = len(german_tokenizer.word_index) + 1
print(f'German vocab size {german_vocab_size}')

eng_tokens = english_tokenizer.texts_to_sequences(eng_data.values)
deu_tokens_output = german_tokenizer.texts_to_sequences(deu_data.values)
deu_tokens_input = german_tokenizer.texts_to_sequences(deu_input_data.values)
english_padded = pad_sequences(eng_tokens, padding='post')
print(f'English padded: {english_padded.shape}')
german_padded_input = pad_sequences(deu_tokens_input, padding='post')
print(f'German input padded: {german_padded_input.shape}')
german_padded_output = pad_sequences(deu_tokens_output, padding='post', maxlen = german_padded_input.shape[1])
print(f'German output padded: {german_padded_output.shape}')

german_padded_output = german_padded_output.reshape(german_padded_output.shape[0], german_padded_output.shape[1], 1)
X_train, X_test, y_train, y_test = train_test_split(np.concatenate((english_padded, german_padded_input), axis =1), german_padded_output, test_size = 7000, shuffle=True, random_state=57)

english_inputs = X_train[:, :101]
print(english_inputs.shape)
german_inputs = X_train[:, 101:]
print(german_inputs.shape)
print(y_train.shape)

english_inputs_val = X_test[:, :101]
print(english_inputs_val.shape)
german_inputs_val = X_test[:, 101:]
print(german_inputs_val.shape)
print(y_test.shape)

English vocab size 16187
German vocab size 34990
English padded: (217032, 101)
German input padded: (217032, 77)
German output padded: (217032, 77)
(210032, 101)
(210032, 77)
(210032, 77, 1)
(7000, 101)
(7000, 77)
(7000, 77, 1)


In [9]:
encoder = Encoder(english_vocab_size, 100, 128)
decoder = Decoder(german_vocab_size, 100, 128)
model = Seq2seq(encoder, decoder)
x1 = np.zeros((1, 101))
x2 = np.zeros((1, 77))
y = np.zeros((1, 77, 1))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=[metrics.sparse_categorical_accuracy])
model.train_on_batch([x1, x2], y)
model.load_weights('/content/drive/My Drive/english-german/0.125.hdf5')

In [92]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=[metrics.sparse_categorical_accuracy])

In [11]:
model.fit(x = [english_inputs, german_inputs], y = y_train,
          batch_size=128, epochs=1,
          callbacks = [ModelCheckpoint(path, monitor='loss', mode='min', save_best_only=True)])

1641/1641 [==============================] - 1110s 677ms/step - loss: 0.1230 - sparse_categorical_accuracy: 0.9728


In [24]:
sentence = 'How are you'.lower()
eng_pred = pad_sequences(english_tokenizer.texts_to_sequences([sentence]), maxlen=101, padding='post')
predict(eng_pred)

'ich sie du du du du du '

In [21]:
def predict(X):
  _, init_states = encoder(X)
  decoder_init = german_tokenizer.word_index['bos'] # token of <BOS> will be first input
  decoder_init = np.array([[decoder_init]]) # Input has to be 2d array

  stop = False
  deu_sentence = ''
  
  while not stop:
    output, next_states = decoder(decoder_init, init_states)
    new_id = np.argmax(output[0, -1, :])
    new_word = german_tokenizer.index_word[new_id]
    
    if (new_word == 'eos' or len(deu_sentence.split()) == 77):
      stop = True
    else:
      deu_sentence += new_word + ' '

    dec_word = np.array([[new_id]])
    init_states = next_states
  return deu_sentence

In [10]:
path = '/content/drive/My Drive/english-german/{loss:.3f}.hdf5'

In [12]:
model.fit(x = [english_inputs, german_inputs], y = y_train,
          batch_size=128, epochs=1,
          callbacks = [ModelCheckpoint(path, monitor='loss', mode='min', save_best_only=True)])

1641/1641 [==============================] - 1111s 677ms/step - loss: 0.1186 - sparse_categorical_accuracy: 0.9737


In [13]:
model.fit(x = [english_inputs, german_inputs], y = y_train,
          batch_size=128, epochs=1,
          callbacks = [ModelCheckpoint(path, monitor='loss', mode='min', save_best_only=True)])

1641/1641 [==============================] - 1110s 677ms/step - loss: 0.1152 - sparse_categorical_accuracy: 0.9743


In [105]:
model.fit(x = [english_inputs, german_inputs], y = y_train,
          batch_size=128, epochs=1,
          callbacks = [ModelCheckpoint(path, monitor='loss', mode='min', save_best_only=True)])

1641/1641 [==============================] - 1130s 689ms/step - loss: 0.3962 - sparse_categorical_accuracy: 0.9339


In [106]:
model.fit(x = [english_inputs, german_inputs], y = y_train,
          batch_size=128, epochs=1,
          callbacks = [ModelCheckpoint(path, monitor='loss', mode='min', save_best_only=True)])

1641/1641 [==============================] - 1130s 689ms/step - loss: 0.3588 - sparse_categorical_accuracy: 0.9386


In [107]:
model.fit(x = [english_inputs, german_inputs], y = y_train,
          batch_size=128, epochs=1,
          callbacks = [ModelCheckpoint(path, monitor='loss', mode='min', save_best_only=True)])

1641/1641 [==============================] - 1130s 688ms/step - loss: 0.3272 - sparse_categorical_accuracy: 0.9428


In [108]:
model.fit(x = [english_inputs, german_inputs], y = y_train,
          batch_size=128, epochs=1,
          callbacks = [ModelCheckpoint(path, monitor='loss', mode='min', save_best_only=True)])

1641/1641 [==============================] - 1130s 688ms/step - loss: 0.3017 - sparse_categorical_accuracy: 0.9461


In [109]:
model.fit(x = [english_inputs, german_inputs], y = y_train,
          batch_size=128, epochs=1,
          callbacks = [ModelCheckpoint(path, monitor='loss', mode='min', save_best_only=True)])

1641/1641 [==============================] - 1130s 689ms/step - loss: 0.2816 - sparse_categorical_accuracy: 0.9486


In [110]:
model.fit(x = [english_inputs, german_inputs], y = y_train,
          batch_size=128, epochs=1,
          callbacks = [ModelCheckpoint(path, monitor='loss', mode='min', save_best_only=True)])

1641/1641 [==============================] - 1131s 689ms/step - loss: 0.2640 - sparse_categorical_accuracy: 0.9507


In [111]:
model.fit(x = [english_inputs, german_inputs], y = y_train,
          batch_size=128, epochs=1,
          callbacks = [ModelCheckpoint(path, monitor='loss', mode='min', save_best_only=True)])

1641/1641 [==============================] - 1131s 689ms/step - loss: 0.2484 - sparse_categorical_accuracy: 0.9527


In [112]:
model.fit(x = [english_inputs, german_inputs], y = y_train,
          batch_size=128, epochs=1,
          callbacks = [ModelCheckpoint(path, monitor='loss', mode='min', save_best_only=True)])

1641/1641 [==============================] - 1130s 689ms/step - loss: 0.2346 - sparse_categorical_accuracy: 0.9545


In [113]:
model.fit(x = [english_inputs, german_inputs], y = y_train,
          batch_size=128, epochs=1,
          callbacks = [ModelCheckpoint(path, monitor='loss', mode='min', save_best_only=True)])

1641/1641 [==============================] - 1129s 688ms/step - loss: 0.2218 - sparse_categorical_accuracy: 0.9563


In [114]:
model.fit(x = [english_inputs, german_inputs], y = y_train,
          batch_size=128, epochs=1,
          callbacks = [ModelCheckpoint(path, monitor='loss', mode='min', save_best_only=True)])

1641/1641 [==============================] - 1129s 688ms/step - loss: 0.2099 - sparse_categorical_accuracy: 0.9580


In [115]:
model.fit(x = [english_inputs, german_inputs], y = y_train,
          batch_size=128, epochs=1,
          callbacks = [ModelCheckpoint(path, monitor='loss', mode='min', save_best_only=True)])

1641/1641 [==============================] - 1129s 688ms/step - loss: 0.1990 - sparse_categorical_accuracy: 0.9597


In [116]:
model.fit(x = [english_inputs, german_inputs], y = y_train,
          batch_size=128, epochs=1,
          callbacks = [ModelCheckpoint(path, monitor='loss', mode='min', save_best_only=True)])

1641/1641 [==============================] - 1126s 686ms/step - loss: 0.1889 - sparse_categorical_accuracy: 0.9612


In [117]:
model.fit(x = [english_inputs, german_inputs], y = y_train,
          batch_size=128, epochs=1,
          callbacks = [ModelCheckpoint(path, monitor='loss', mode='min', save_best_only=True)])

1641/1641 [==============================] - 1119s 682ms/step - loss: 0.1797 - sparse_categorical_accuracy: 0.9628


In [118]:
model.fit(x = [english_inputs, german_inputs], y = y_train,
          batch_size=128, epochs=1,
          callbacks = [ModelCheckpoint(path, monitor='loss', mode='min', save_best_only=True)])

1641/1641 [==============================] - 1122s 683ms/step - loss: 0.1715 - sparse_categorical_accuracy: 0.9641


In [119]:
model.fit(x = [english_inputs, german_inputs], y = y_train,
          batch_size=128, epochs=1,
          callbacks = [ModelCheckpoint(path, monitor='loss', mode='min', save_best_only=True)])

1641/1641 [==============================] - 1122s 684ms/step - loss: 0.1638 - sparse_categorical_accuracy: 0.9654


In [120]:
model.fit(x = [english_inputs, german_inputs], y = y_train,
          batch_size=128, epochs=1,
          callbacks = [ModelCheckpoint(path, monitor='loss', mode='min', save_best_only=True)])

1641/1641 [==============================] - 1119s 682ms/step - loss: 0.1570 - sparse_categorical_accuracy: 0.9666


In [121]:
model.fit(x = [english_inputs, german_inputs], y = y_train,
          batch_size=128, epochs=1,
          callbacks = [ModelCheckpoint(path, monitor='loss', mode='min', save_best_only=True)])

1641/1641 [==============================] - 1113s 678ms/step - loss: 0.1506 - sparse_categorical_accuracy: 0.9677


In [122]:
model.fit(x = [english_inputs, german_inputs], y = y_train,
          batch_size=128, epochs=1,
          callbacks = [ModelCheckpoint(path, monitor='loss', mode='min', save_best_only=True)])

1641/1641 [==============================] - 1107s 674ms/step - loss: 0.1447 - sparse_categorical_accuracy: 0.9688


In [123]:
model.fit(x = [english_inputs, german_inputs], y = y_train,
          batch_size=128, epochs=1,
          callbacks = [ModelCheckpoint(path, monitor='loss', mode='min', save_best_only=True)])

1641/1641 [==============================] - 1108s 675ms/step - loss: 0.1393 - sparse_categorical_accuracy: 0.9698


In [124]:
model.fit(x = [english_inputs, german_inputs], y = y_train,
          batch_size=128, epochs=1,
          callbacks = [ModelCheckpoint(path, monitor='loss', mode='min', save_best_only=True)])

1641/1641 [==============================] - 1108s 675ms/step - loss: 0.1344 - sparse_categorical_accuracy: 0.9707


In [125]:
model.fit(x = [english_inputs, german_inputs], y = y_train,
          batch_size=128, epochs=1,
          callbacks = [ModelCheckpoint(path, monitor='loss', mode='min', save_best_only=True)])

1641/1641 [==============================] - 1106s 674ms/step - loss: 0.1298 - sparse_categorical_accuracy: 0.9715


In [126]:
model.fit(x = [english_inputs, german_inputs], y = y_train,
          batch_size=128, epochs=1,
          callbacks = [ModelCheckpoint(path, monitor='loss', mode='min', save_best_only=True)])

1641/1641 [==============================] - 1107s 675ms/step - loss: 0.1255 - sparse_categorical_accuracy: 0.9723


In [ ]:
model.fit(x = [english_inputs, german_inputs], y = y_train,
          batch_size=128, epochs=1,
          callbacks = [ModelCheckpoint(path, monitor='loss', mode='min', save_best_only=True)])

 874/1641 [==============>...............] - ETA: 8:37 - loss: 0.1198 - sparse_categorical_accuracy: 0.9736

In [ ]:
model.fit(x = [english_inputs, german_inputs], y = y_train,
          batch_size=128, epochs=1,
          callbacks = [ModelCheckpoint(path, monitor='loss', mode='min', save_best_only=True)])

In [ ]:
model.fit(x = [english_inputs, german_inputs], y = y_train,
          batch_size=128, epochs=1,
          callbacks = [ModelCheckpoint(path, monitor='loss', mode='min', save_best_only=True)])

In [ ]:
model.fit(x = [english_inputs, german_inputs], y = y_train,
          batch_size=128, epochs=1,
          callbacks = [ModelCheckpoint(path, monitor='loss', mode='min', save_best_only=True)])

In [ ]:
model.fit(x = [english_inputs, german_inputs], y = y_train,
          batch_size=128, epochs=1,
          callbacks = [ModelCheckpoint(path, monitor='loss', mode='min', save_best_only=True)])

In [ ]:
model.fit(x = [english_inputs, german_inputs], y = y_train,
          batch_size=128, epochs=1,
          callbacks = [ModelCheckpoint(path, monitor='loss', mode='min', save_best_only=True)])

In [ ]:
model.fit(x = [english_inputs, german_inputs], y = y_train,
          batch_size=128, epochs=1,
          callbacks = [ModelCheckpoint(path, monitor='loss', mode='min', save_best_only=True)])

In [ ]:
model.save('/content/drive/My Drive/english-german/diff_model.hdf5')

In [ ]:
model.save('/content/drive/My Drive/english-german/diff_model.hdf5')